In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7500)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)
        
print('GPU ready')

In [1]:
import pandas as pd

category_csv = pd.read_csv('./category/category.csv', encoding='CP949')
label_dict = dict(category_csv[['bottle_name', 'bottle_id']].values)
print(label_dict)

{'wine': 0, 'soju': 1, 'plate': 2, 'me': 3, 'glass': 4, 'energydrink': 5, 'case': 6, 'bottle': 7, 'be': 8}


In [130]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def Edge_detection(image):
    return cv2.Canny(image.astype(np.uint8), 50, 150)

In [131]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

rootPath = './image_data/촬영본'

with tf.device('/device:GPU:0'):

    
    
    train_imageGenerator = ImageDataGenerator(
        preprocessing_function=Edge_detection,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
#         zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=.2
    )
    
    val_imageGenerator = ImageDataGenerator(
        preprocessing_function=Edge_detection,
        rescale=1./255,
        validation_split=.2
    )
    
    trainGen = train_imageGenerator.flow_from_directory(
        rootPath,
        target_size=(299, 299),
        classes=label_dict,
        subset='training',
        batch_size = 100,
        class_mode="sparse"
    )

    validationGen = val_imageGenerator.flow_from_directory(
        rootPath,
        target_size=(299, 299),
        classes=label_dict,
        subset='validation',
        batch_size = 20,
        class_mode="sparse"
    )
    train_batch_size =100
    val_batch_size =20


    print(len(trainGen.classes))
    print(len(validationGen.classes))



Found 100 images belonging to 9 classes.
Found 20 images belonging to 9 classes.
100
20


In [132]:

with tf.device('/device:GPU:0'):
    x_train, y_train= validationGen.next()
    


#     img = x_train[0]*255
#     img = img.astype(np.uint8)
    plt.imshow(x_train[0])
#     img_gray = cv2.cvtColor(img, cv2.IMREAD_GRAYSCALE)
#     plt.imshow(img_gray)
#     img_canny = cv2.Canny(img_gray, 50, 150)
#     plt.imshow(img_canny)

UFuncTypeError: Cannot cast ufunc 'multiply' output from dtype('float64') to dtype('uint8') with casting rule 'same_kind'

In [135]:
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dense, InputLayer,ZeroPadding2D, ZeroPadding1D 
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, InputLayer
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



with tf.device('/device:GPU:0'):

    model = Sequential()
    model.add(InputLayer(input_shape=(299,299)))
    
#     model.add(ZeroPadding2D(padding=(2, 2)))
#     model.add(Conv2D(96, 
#                      kernel_size = (11,11), 
#                      strides=4, 
#                      ))

#     model.add(MaxPooling2D(pool_size=(3, 3)))
    
    
#     model.add(ZeroPadding2D(padding=(2, 2)))
#     model.add(Conv2D(192, 
#                      kernel_size = (5,5), 
#                      strides=1,))

#     model.add(MaxPooling2D(pool_size=(3, 3)))

#     model.add(ZeroPadding2D(padding=(1, 1)))
#     model.add(Conv2D(288, 
#                      kernel_size = (3,3), 
#                      strides=1))
    
#     model.add(ZeroPadding2D(padding=(1, 1)))
#     model.add(Conv2D(288, 
#                      kernel_size = (3,3), 
#                      strides=1,
#                      ))

#     model.add(ZeroPadding2D(padding=(1, 1)))
#     model.add(Conv2D(192, 
#                      kernel_size = (3,3), 
#                      strides=1,))

#     model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

    model.add(Flatten(input_shape=(299*299,)))
    
    model.add(Dense(4096, activation='relu'))

#     model.add(Dense(4096, activation='relu'))
    
    model.add(Dense(9, activation='softmax'))

    model.summary()

    model.compile(optimizer=RMSprop(learning_rate=1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_accuracy'])
    # Model saving callback
    checkpointer = ModelCheckpoint(filepath='./model/recycle_Tung_Yang_Canny.h5', monitor='val_loss', verbose=1, save_best_only=True)

#     Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', verbose=1, patience=10)

    epochs = 500
    history = model.fit_generator(
        trainGen, 
        epochs=epochs,
        callbacks=[checkpointer, early_stopping],
        steps_per_epoch=1, 
        validation_data=validationGen,
        validation_steps=1,
        verbose=1
    )


ResourceExhaustedError: OOM when allocating tensor with shape[89401,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:

import matplotlib.pyplot as plt



train_acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']

plt.plot(train_acc,'bo', color='r', label='accuracy')
plt.plot(val_acc,'b', color='b', label='val accuracy')
plt.legend()
plt.show()


train_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(train_loss,'bo', color='r', label='train loss')
plt.plot(val_loss,'b', color='b', label='validation loss')
plt.legend()
plt.show()

In [ ]:
#### from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns 
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

with tf.device('/device:GPU:0'):


    model_path = './model/recycle_Tung_Yang_Canny.h5'


    vgg_model = tf.keras.models.load_model(model_path)


    x_train, y_train= validationGen.next()
    print(y_train)

    #Confution Matrix and Classification Report
    Y_pred = vgg_model.predict_generator(validationGen, 1)

    y_pred = np.argmax(Y_pred, axis=1)


    print('Confusion Matrix')
    # display(confusion_matrix(validationGen.classes, y_pred))

    print(y_pred)





    fig, ax = plt.subplots(figsize=(10,10))  # inch단위로 그림의 크기
    sns.heatmap(
        confusion_matrix(y_train, y_pred),
        annot = True, # 숫자표현
        fmt = '3d',    # 정수표현     
        cmap = 'Blues', # color 색상
        ax = ax,       # 그래프로 사용할 subplot
        xticklabels=category_csv['bottle_name'], 
        yticklabels=category_csv['bottle_name']
    )
    ax.set_xlabel('Predict')
    ax.set_ylabel('Actual')
    plt.show()

